# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fcd7c9c5f70>
├── 'a' --> tensor([[-1.2265, -1.0759, -0.8193],
│                   [-0.1557,  0.2285,  0.8063]])
└── 'x' --> <FastTreeValue 0x7fcd7c9c5e50>
    └── 'c' --> tensor([[ 0.3738,  0.0561,  1.8010, -2.2632],
                        [ 1.0539,  0.7086,  0.0033,  1.3589],
                        [-0.3100,  0.1563, -0.6300, -0.4081]])

In [4]:
t.a

tensor([[-1.2265, -1.0759, -0.8193],
        [-0.1557,  0.2285,  0.8063]])

In [5]:
%timeit t.a

71.1 ns ± 0.0772 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fcd7c9c5f70>
├── 'a' --> tensor([[ 0.8809, -0.2469, -0.5030],
│                   [-2.5372,  2.0406,  1.1537]])
└── 'x' --> <FastTreeValue 0x7fcd7c9c5e50>
    └── 'c' --> tensor([[ 0.3738,  0.0561,  1.8010, -2.2632],
                        [ 1.0539,  0.7086,  0.0033,  1.3589],
                        [-0.3100,  0.1563, -0.6300, -0.4081]])

In [7]:
%timeit t.a = new_value

62.3 ns ± 0.0168 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.2265, -1.0759, -0.8193],
               [-0.1557,  0.2285,  0.8063]]),
    x: Batch(
           c: tensor([[ 0.3738,  0.0561,  1.8010, -2.2632],
                      [ 1.0539,  0.7086,  0.0033,  1.3589],
                      [-0.3100,  0.1563, -0.6300, -0.4081]]),
       ),
)

In [10]:
b.a

tensor([[-1.2265, -1.0759, -0.8193],
        [-0.1557,  0.2285,  0.8063]])

In [11]:
%timeit b.a

72.2 ns ± 0.06 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.0951,  0.7106,  0.7091],
               [-0.6114, -0.0332, -0.8371]]),
    x: Batch(
           c: tensor([[ 0.3738,  0.0561,  1.8010, -2.2632],
                      [ 1.0539,  0.7086,  0.0033,  1.3589],
                      [-0.3100,  0.1563, -0.6300, -0.4081]]),
       ),
)

In [13]:
%timeit b.a = new_value

491 ns ± 0.142 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

845 ns ± 0.067 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 52.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 418 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 511 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fcccc8d1f70>
├── 'a' --> tensor([[[-1.2265, -1.0759, -0.8193],
│                    [-0.1557,  0.2285,  0.8063]],
│           
│                   [[-1.2265, -1.0759, -0.8193],
│                    [-0.1557,  0.2285,  0.8063]],
│           
│                   [[-1.2265, -1.0759, -0.8193],
│                    [-0.1557,  0.2285,  0.8063]],
│           
│                   [[-1.2265, -1.0759, -0.8193],
│                    [-0.1557,  0.2285,  0.8063]],
│           
│                   [[-1.2265, -1.0759, -0.8193],
│                    [-0.1557,  0.2285,  0.8063]],
│           
│                   [[-1.2265, -1.0759, -0.8193],
│                    [-0.1557,  0.2285,  0.8063]],
│           
│                   [[-1.2265, -1.0759, -0.8193],
│                    [-0.1557,  0.2285,  0.8063]],
│           
│                   [[-1.2265, -1.0759, -0.8193],
│                    [-0.1557,  0.2285,  0.8063]]])
└── 'x' --> <FastTreeValue 0x7fccc5cb7610>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.5 µs ± 61.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fcccc8d1ac0>
├── 'a' --> tensor([[-1.2265, -1.0759, -0.8193],
│                   [-0.1557,  0.2285,  0.8063],
│                   [-1.2265, -1.0759, -0.8193],
│                   [-0.1557,  0.2285,  0.8063],
│                   [-1.2265, -1.0759, -0.8193],
│                   [-0.1557,  0.2285,  0.8063],
│                   [-1.2265, -1.0759, -0.8193],
│                   [-0.1557,  0.2285,  0.8063],
│                   [-1.2265, -1.0759, -0.8193],
│                   [-0.1557,  0.2285,  0.8063],
│                   [-1.2265, -1.0759, -0.8193],
│                   [-0.1557,  0.2285,  0.8063],
│                   [-1.2265, -1.0759, -0.8193],
│                   [-0.1557,  0.2285,  0.8063],
│                   [-1.2265, -1.0759, -0.8193],
│                   [-0.1557,  0.2285,  0.8063]])
└── 'x' --> <FastTreeValue 0x7fccc5c3ea30>
    └── 'c' --> tensor([[ 0.3738,  0.0561,  1.8010, -2.2632],
                        [ 1.0539,  0.7086,  0.0033,  1.3589],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 90.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.1 µs ± 48.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.3738,  0.0561,  1.8010, -2.2632],
                       [ 1.0539,  0.7086,  0.0033,  1.3589],
                       [-0.3100,  0.1563, -0.6300, -0.4081]],
              
                      [[ 0.3738,  0.0561,  1.8010, -2.2632],
                       [ 1.0539,  0.7086,  0.0033,  1.3589],
                       [-0.3100,  0.1563, -0.6300, -0.4081]],
              
                      [[ 0.3738,  0.0561,  1.8010, -2.2632],
                       [ 1.0539,  0.7086,  0.0033,  1.3589],
                       [-0.3100,  0.1563, -0.6300, -0.4081]],
              
                      [[ 0.3738,  0.0561,  1.8010, -2.2632],
                       [ 1.0539,  0.7086,  0.0033,  1.3589],
                       [-0.3100,  0.1563, -0.6300, -0.4081]],
              
                      [[ 0.3738,  0.0561,  1.8010, -2.2632],
                       [ 1.0539,  0.7086,  0.0033,  1.3589],
                       [-0.3100,  0.1563, -0.6300, -0.4081]],

In [26]:
%timeit Batch.stack(batches)

78.4 µs ± 266 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.3738,  0.0561,  1.8010, -2.2632],
                      [ 1.0539,  0.7086,  0.0033,  1.3589],
                      [-0.3100,  0.1563, -0.6300, -0.4081],
                      [ 0.3738,  0.0561,  1.8010, -2.2632],
                      [ 1.0539,  0.7086,  0.0033,  1.3589],
                      [-0.3100,  0.1563, -0.6300, -0.4081],
                      [ 0.3738,  0.0561,  1.8010, -2.2632],
                      [ 1.0539,  0.7086,  0.0033,  1.3589],
                      [-0.3100,  0.1563, -0.6300, -0.4081],
                      [ 0.3738,  0.0561,  1.8010, -2.2632],
                      [ 1.0539,  0.7086,  0.0033,  1.3589],
                      [-0.3100,  0.1563, -0.6300, -0.4081],
                      [ 0.3738,  0.0561,  1.8010, -2.2632],
                      [ 1.0539,  0.7086,  0.0033,  1.3589],
                      [-0.3100,  0.1563, -0.6300, -0.4081],
                      [ 0.3738,  0.0561,  1.8010, -2.2632],
                   

In [28]:
%timeit Batch.cat(batches)

161 µs ± 268 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 1.44 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
